System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.10.2
Commit bd47eca2c8a (2024-03-01 10:14 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (x86_64-apple-darwin22.4.0)
  CPU: 12 × Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, skylake)
Threads: 1 default, 0 interactive, 1 GC (on 12 virtual cores)


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/GitHub/biostat-m257-2024-spring/hw3/hw3`


Status `~/Documents/GitHub/biostat-m257-2024-spring/hw3/hw3/Project.toml`
  [6e4b80f9] BenchmarkTools v1.5.0
  [31c24e10] Distributions v0.25.108
  [7522ee7d] SweepOperator v0.3.4
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


In [3]:
using LinearAlgebra, Random
using BenchmarkTools, Distributions

Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$. 

### Solution

\begin{align}
    log-likelihood_i = -\frac{n_i}{2}log(2\pi) - \frac{1}{2}log\big[det(\mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T + \sigma^2 I)\big] - \frac{1}{2}(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})^T(\mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T + \sigma^2 I)^{-1} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})
\end{align}

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

In [4]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate vectors/arrays you may want to pre-allocate
    storage_p  :: Vector{T}
    xty        :: Vector{T}
    storage_q  :: Vector{T}
    zty        :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    yty        :: T
    storage_qq :: Matrix{T}
    storage_qq2 :: Matrix{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
        ) where T <: AbstractFloat
    storage_p  = Vector{T}(undef, size(X, 2))
    xty        = transpose(X) * y
    storage_q  = Vector{T}(undef, size(Z, 2))
    zty        = transpose(Z) * y
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    yty        = transpose(y) * y
    storage_qq = similar(ztz) #used to storage inverse(σ²I + L^T Z_i^T Z_i L)
    storage_qq2 = similar(ztz) #used to storage inverse(σ²I + L^T Z_i^T Z_i L)
    LmmObs(y, X, Z, storage_p, xty, storage_q, zty, xtx, ztx, ztz, yty, storage_qq, storage_qq2)
end

LmmObs

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.  

### Solution:

Two formulas can be used to simplify the computations.

1.

$$
(I + AB)^{-1} = I - A(I + BA)^{-1}B
$$

2.

$$
det(I + AB) = det(I + BA)
$$

Mathematical proof:

\begin{align}
l\left(\beta, \Sigma, \sigma^{2}\right)
& =-\frac{n}{2} \log (2 \pi)-\frac{1}{2} \log |V|-\frac{1}{2}(Y-X \beta)^{T} V^{-1}(Y-X \beta) 
\\
& =-\frac{n}{2} \log (2 \pi)-\frac{1}{2} \log \left|Z LL^T Z^T+\sigma^{2} I_{n}\right|-\frac{1}{2}\left(Y-X(\beta)^{T}\left(Z LL^T Z^{T}+\sigma^{2} I_{n}\right)^{-1}(Y-x \beta)\right)
\\
& =-\frac{n}{2} \log (2 \pi) - \frac{n}{2} \log  \sigma^{2} - \frac{1}{2} \log \left|I + L^TZ^{T}ZL / \sigma^{2}\right| -\frac{1}{2 \sigma^{2}}(Y-x \beta)^{T}(Y-x \beta)
\\
& \quad +\frac{1}{2 \sigma^{2}}(Y-x \beta)^{T} ZL\big(I+ L^T Z^{T} Z L / \sigma^{2}\big)^{-1}L^T Z^{T}(Y-x \beta) 
\\
& =-\frac{n}{2} \log (2 \pi\sigma^{2}) -\frac{1}{2} \log \left|I + L^TZ^{T}ZL / \sigma^{2}\right| -\frac{1}{2 \sigma^{2}}(Y-x \beta)^{T}(Y-x \beta)
\\
&\quad+\frac{1}{2 \sigma^{4}}\left(Z^{T} Y-Z^{T}x \beta\right)^{T}L\left(I+ L^TZ^T ZL /\sigma^{2}\right)^{-1}L^T\left(Z^{T}Y-Z^{T}x \beta\right)
\\
& =-\frac{n}{2} \log (2 \pi\sigma^{2}) -\frac{1}{2} \log \left|L_2 L_2^T\right| -\frac{1}{2 \sigma^{2}}(Y-x \beta)^{T}(Y-x \beta)
\\
&\quad+\frac{1}{2 \sigma^{4}}\left(Z^{T} Y-Z^{T}x \beta\right)^{T}L\left(L_2 L_2^T\right)^{-1}L^T\left(Z^{T}Y-Z^{T}x \beta\right)
\\
& =-\frac{n}{2} \log (2 \pi\sigma^{2}) -\frac{1}{2} \log \left|L_2 L_2^T\right| -\frac{1}{2 \sigma^{2}}(Y-x \beta)^{T}(Y-x \beta)+\frac{1}{2 \sigma^{4}}\left|L_2^{-1}\left(L^TZ^{T} Y- L^TZ^{T}x \beta\right)\right|^2_2
\\
& =-\frac{n}{2} \log (2 \pi\sigma^{2}) - \log \left|L_2\right| -\frac{1}{2 \sigma^{2}}(Y-x \beta)^{T}(Y-x \beta)+\frac{1}{2 \sigma^{4}}\left|L_2^{-1}\left(L^TZ^{T} Y - L^TZ^{T}x \beta\right)\right|^2_2
\end{align}

where $L_2$ is the Cholesky decomposition result of $I + L^TZ^TZL/\sigma^2$

In [5]:
function logl1!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    
    BLAS.symm!('L', 'U', 1 / σ², obs.ztz, L, 0.0, obs.storage_qq)
    BLAS.gemm!('T', 'N', 1.0, L, obs.storage_qq, 0.0, obs.storage_qq2)
    obs.storage_qq .= Symmetric(obs.storage_qq2) + I(q)
    #Prepare for the derterminant part
    #BLAS.syrk!('U', 'T', 1.0, Linv, 1 / σ², obs.storage_qq)
    #obs.storage_qq .= Symmetric(obs.storage_qq)
    LAPACK.potrf!('L', obs.storage_qq)
    obs.storage_qq .= LowerTriangular(obs.storage_qq)
    #Prepare for Z^Ty - Z^TX * β
    obs.storage_q .= obs.zty
    BLAS.gemv!('N', -1.0, obs.ztx, β, 1.0, obs.storage_q)
    obs.storage_q .= transpose(L) * obs.storage_q
    #Prepare for the inverse 
    BLAS.trsv!('L', 'N', 'N', obs.storage_qq, obs.storage_q)
    #Prepare for X^TX * β
    BLAS.gemv!('N', 1.0, obs.xtx, β, 0.0, obs.storage_p)
    return(- (n//2) * log(2π) - (n//2) * log(σ²) 
        - (sum(log.(diag(obs.storage_qq))))
        - (1//2) * (1 / σ²) * (obs.yty - 2 * dot(obs.xty, β) + dot(β, obs.storage_p) 
            - dot(obs.storage_q, obs.storage_q) * (1/σ²)
        )
    )   
end

logl1! (generic function with 1 method)

**If it is allowed to precompute the inverse of L, then the running speed and memory usage can be further promoted.**

\begin{align}
l\left(\beta, \Sigma, \sigma^{2}\right)
& =-\frac{n}{2} \log (2 \pi)-\frac{1}{2} \log |V|-\frac{1}{2}(Y-X \beta)^{T} V^{-1}(Y-X \beta) 
\\
& =-\frac{n}{2} \log (2 \pi)-\frac{1}{2} \log \left|Z LL^T Z^T+\sigma^{2} I_{n}\right|-\frac{1}{2}\left(Y-X(\beta)^{T}\left(Z LL^T Z^{T}+\sigma^{2} I_{n}\right)^{-1}(Y-x \beta)\right)
\\
& =-\frac{n}{2} \log (2 \pi) - \frac{n}{2} \log  \sigma^{2} - \frac{1}{2} \log |L^T| \left|(LL^T)^{-1} + Z^{T}Z / \sigma^{2}\right| |L| 
\\
& \quad -\frac{1}{2 \sigma^{2}}(Y-x \beta)^{T}(Y-x \beta)+\frac{1}{2 \sigma^{2}}(Y-x \beta)^{T} Z\big((LL^T)^{-1}+Z^{T} Z / \sigma^{2}\big)^{-1} Z^{T}(Y-x \beta) 
\\
& =-\frac{n}{2} \log (2 \pi\sigma^{2}) -\frac{1}{2} \log |L^T| \left|(LL^T)^{-1} + Z^{T}Z / \sigma^{2}\right| |L| -\frac{1}{2 \sigma^{2}}(Y-x \beta)^{T}(Y-x \beta)
\\
&\quad+\frac{1}{2 \sigma^{4}}\left(Z^{T} Y-Z^{T}x \beta\right)^{T}\left((LL^T)^{-1}+Z^{T} Z /\sigma^{2}\right)^{-1}\left(Z^{T}Y-Z^{T}x \beta\right)
\\
& =-\frac{n}{2} \log (2 \pi\sigma^{2}) -\frac{1}{2} \log |L^T| \left|L_2 L_2^T\right| |L| -\frac{1}{2 \sigma^{2}}(Y-x \beta)^{T}(Y-x \beta)
\\
&\quad+\frac{1}{2 \sigma^{4}}\left(Z^{T} Y-Z^{T}x \beta\right)^{T}\left(L_2 L_2^T\right)^{-1}\left(Z^{T}Y-Z^{T}x \beta\right)
\\
& =-\frac{n}{2} \log (2 \pi\sigma^{2}) -\frac{1}{2} \log |L^T| \left|L_2 L_2^T\right| |L| -\frac{1}{2 \sigma^{2}}(Y-x \beta)^{T}(Y-x \beta)+\frac{1}{2 \sigma^{4}}\left|L_2^{-1}\left(Z^{T} Y-Z^{T}x \beta\right)\right|^2_2
\\
& =-\frac{n}{2} \log (2 \pi\sigma^{2}) + \log \left|L^{-1}\right| - \log \left|L_2\right| -\frac{1}{2 \sigma^{2}}(Y-x \beta)^{T}(Y-x \beta)+\frac{1}{2 \sigma^{4}}\left|L_2^{-1}\left(Z^{T} Y-Z^{T}x \beta\right)\right|^2_2
\end{align}

where $L_2$ is the Cholesky decomposition result of $(LL^T)^{-1} + Z^TZ/\sigma^2 = (L^{-1})^TL^{-1}+ Z^TZ/\sigma^2$

In [6]:
function logl2!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        Linv:: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    

    obs.storage_qq .= obs.ztz
    #Prepare for the derterminant part
    BLAS.syrk!('U', 'T', 1.0, Linv, 1 / σ², obs.storage_qq)
    obs.storage_qq .= Symmetric(obs.storage_qq)
    LAPACK.potrf!('L', obs.storage_qq)
    obs.storage_qq .= LowerTriangular(obs.storage_qq)
    #Prepare for Z^Ty - Z^TX * β
    obs.storage_q .= obs.zty
    BLAS.gemv!('N', -1.0, obs.ztx, β, 1.0, obs.storage_q)
    #Prepare for the inverse 
    BLAS.trsv!('L', 'N', 'N', obs.storage_qq, obs.storage_q)
    #Prepare for X^TX * β
    BLAS.gemv!('N', 1.0, obs.xtx, β, 0.0, obs.storage_p)
    return(- (n//2) * log(2π) - (n//2) * log(σ²) 
        - (sum(log.(diag(obs.storage_qq))) - sum(log.(diag(Linv))))
        - (1//2) * (1 / σ²) * (obs.yty - 2 * dot(obs.xty, β) + dot(β, obs.storage_p) 
            - dot(obs.storage_q, obs.storage_q) * (1/σ²)
        )
    )   
end

logl2! (generic function with 1 method)

**Hint**: This function shouldn't be very long. Mine, obeying 92-character rule, is 30 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm (flop count) first then use BLAS functions to reduce memory allocations.

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

### Solution

In [26]:
Random.seed!(257)

# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 2.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

LmmObs{Float64}([-0.954332745798538, 1.708628623631947, 6.158974576749139, 5.169631703045298, 0.6809423111307418, 2.959889369862154, 0.42003005461150134, 4.093957900991921, 5.948403040653879, 2.8053988068901172  …  4.21150686507263, -0.9916538074359607, -1.3881101188876772, -0.8404895572053244, 8.326165884151456, 4.050730156446721, 6.7987277486925235, 4.083908668925583, 1.1264587302316045, 6.230980152631514], [1.0 0.6790633442371218 … 0.5400611947971554 -0.632040682052606; 1.0 1.2456776800889142 … -0.4818455756130373 0.6467830314674976; … ; 1.0 0.0733124748775436 … 0.6125080259511859 0.4181258283983667; 1.0 -1.336609049786048 … -0.18567490803712938 1.0745977099307227], [1.0 -1.0193326822839996 -0.15855601254314888; 1.0 1.7462667837699666 -0.4584376230657152; … ; 1.0 1.4843185594903878 0.42458303115266854; 1.0 0.3791714762820068 0.25150666970865837], [8.094e-320, 1.0e-323, 2.8135696016e-314, 2.813569633e-314, 0.0], [4370.744591901516, -1741.6337781007833, 1263.9024057666916, 303.7688688

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [27]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3257.87179442897

Check that your answer matches that from Distributions.jl

In [28]:
L = Matrix(cholesky(Σ).L)
logl1!(obs, β, L, σ²)

-3257.8717944289647

In [29]:
L = Matrix(cholesky(Σ).L)
Linv = inv(L)
logl2!(obs, β, Linv, σ²)

-3257.871794428965

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertionError`.

In [30]:
@assert logl1!(obs, β, L, σ²) ≈ logpdf(mvn, y)

In [31]:
@assert logl2!(obs, β, Linv, σ²) ≈ logpdf(mvn, y)

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

### Solution:

In [13]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 5992 samples with 1 evaluation.
 Range (min … max):  740.934 μs …  11.537 ms  ┊ GC (min … max): 0.00% … 91.84%
 Time  (median):     808.494 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   827.357 μs ± 196.734 μs  ┊ GC (mean ± σ):  0.40% ±  1.67%

       █▄██▆▄▅▅▃▁                                                
  ▂▃▆▅▆██████████▇▇▆▆▅▄▄▃▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  741 μs           Histogram: frequency by time         1.12 ms <

 Memory estimate: 31.52 KiB, allocs estimate: 3.

In [14]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl1!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.055 μs …   8.635 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.160 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.249 μs ± 241.354 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▂▁ ▃█▄                                                      
  ▅██▇███▇█▆▄▂▂▂▁▁▂▁▁▄▆▅▄▅▄▅▄▃▃▂▂▃▅▄▄▂▃▄▃▂▂▄▆▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁ ▃
  1.06 μs         Histogram: frequency by time        1.67 μs <

 Memory estimate: 432 bytes, allocs estimate: 5.

In [15]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
Linv = inv(L)
bm3 = @benchmark logl2!($obs, $β, $Linv, $σ²)

BenchmarkTools.Trial: 10000 samples with 77 evaluations.
 Range (min … max):  829.870 ns …  39.307 μs  ┊ GC (min … max): 0.00% … 97.11%
 Time  (median):     838.688 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   876.369 ns ± 544.791 ns  ┊ GC (mean ± σ):  0.86% ±  1.37%

  █▆ ▇▇  ▁                        ▃               ▂▁    ▁       ▂
  ██████▇█▇▆▆▆▁▁▅▅▃▁▆▄▁▃▅▅▃▁▁▆▅▅▅██▇▄▆▃▃▄▃▅▆█▇▄▅▅▆██▇▅▅▆██▆▅▅▆▇ █
  830 ns        Histogram: log(frequency) by time       1.22 μs <

 Memory estimate: 320 bytes, allocs estimate: 4.

The points you will get is
$$
\frac{x}{1000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [16]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

20.914736569802535

In [17]:
# this is the points you'll get
clamp(median(bm1).time / median(bm3).time / 1000 * 30, 0, 30)

28.919945183418758

**Hint**: Apparently I am using 1000 as denominator because I expect your code to be at least $1000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

### Solution

In [18]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

29.578125

In [19]:
clamp(30 - median(bm3).memory / 1024, 0, 30)

29.6875

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.